# Trabalho Prático 2 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Criptosistemas pós-quânticos PKE/KEM* - Kyber/Crystalis

Implementar o esquema Kyber/Crystalis em classes *Python/SageMath* apresentando, para cada um, as versões **KEM-IND-CPA** e **PKE-IND-CCA**.

Nota: Baseado no documento Kyber/Crystalis (https://pq-crystals.org/kyber/data/kyber-specification-round3.pdf)

### Kyber/Crystalis - KEM-IND-CPA

In [2]:
import math, os, numpy as np
from hashlib import sha3_512 as G, shake_128 as XOF, sha3_256 as H, shake_256 as PRF

'''
Parâmetros definidos para a implementação do kyber
'''
n = 256
_n = 9
q = 7681
Qq = PolynomialRing(GF(q), 'x')
y = Qq.gen()
RQ = QuotientRing(Qq, y^n+1)

'''
Definição da função mod+-
'''
def modMm(r,a) :
    _r = r % a
    # Testar se a é par
    if mod(a,2)==0 :
        # Cálculo dos limites -a/2 e a/2
        inf_bound, sup_bound = -a/2, a/2
    # a é ímpar
    else :
        # Cálculo dos limites -a-1/2 e a-1/2
        inf_bound, sup_bound = (-a-1)/2, (a-1)/2
    # Queremos garantir que o módulo se encontre no intervalo calculado
    while _r > sup_bound :
        _r-=a
    while _r < inf_bound :
        _r+=a
    return _r
    
    

'''
Função que converte um array de bytes num array de bits
'''
def bytesToBits(bytearr) :
    bitarr = []
    for elem in bytearr :
        bitElemArr = []
        # Calculamos cada bit pertencente ao byte respetivo
        for i in range(0,8) :
            bitElemArr.append(mod(elem // 2**(mod(i,8)),2))
        
        for i in range(0,len(bitElemArr)) :
            bitarr.append(bitElemArr[i])
    return bitarr

'''
Função que converte um array de bits num array de bytes
'''
def bitsToBytes(bitarr) :
    bytearr = []
    bit_arr_size = len(bitarr)
    byte_arr_size = bit_arr_size / 8
    for i in range(byte_arr_size) :
        elem = 0
        for j in range(8) : # Definir macro BYTE_SIZE = 0
            elem += (int(bitarr[i*8+j]) * 2**j)
        bytearr.append(elem)
    return bytearr
    
        


''' 
Função parse cuja finalidade é receber como input 
uma byte stream e retornar, como output, a representação NTT
'''
def parse(b) :
    coefs = [0]*n # O poly terá n=256 coeficientes
    i,j = 0,0
    while j<n :
        d = b[i] + 256*b[i+1]
        d = mod(d,2**13)
        if d<q :
            coefs[br(8,j)] = d
            j+=1
        i+=2
    return RQ(coefs)

'''
Implementação da função que implementa 
o bit reversed order

Parâmetros:
    - _bits : nº de bits usados para representar nr
    - nr : valor a ser bitreversed
'''
def br(_bits, nr) :
    res = 0
    for i in range(_bits) :
        res += (nr % 2) * 2**(_bits-i-1)
        nr = nr // 2
    return res
    

'''
Definição da função compress
'''
def compress(q,x,d) :
    return mod(round((2**d)/q * int(x)),2**d)

'''
Definição da função decompress
'''
def decompress(q,x,d) :
    return round((q/2**d) * ZZ(x))


'''
Definição da função CBD. Recebe como 
input o n (comprido) e o array de bytes
'''
def cbd(noise, btarray) :
    f = []
    bitArray = bytesToBits(btarray)
    for i in range(256) :
        a, b = 0, 0
        # Cálculo do a e do b
        for j in range(256) :
            a+=bitArray[2*i*noise + j]
            b+=bitArray[2*i*noise + noise + j]
        f.append(a - b)
    return RQ(f)


'''
Implementação da função decode
'''
def decode(l, btarray) :
    f = []
    bitArray = bytesToBits(btarray)
    for i in range(256) :
        fi = 0
        for j in range(l) :
            fi += int(bitArray[i*l+j]) * 2**j
        f.append(fi)
    return RQ(f)

'''
Implementação da função encode
'''
def encode(l, poly) :
    bitArr = []
    coef_array = poly.list()
    # Percorremos cada coeficiente
    for i in range(256) :
        actual = int(coef_array[i])
        for j in range(l) :
            bitArr.append(actual % 2)
            actual = actual // 2
    return bitsToBytes(bitArr)

'''
Implementação da função NTT
'''
def ntt(g) :
    
    w, psi = 3844, 62 # Parâmetros definidos no documento da especificação do kyber
    
    '''
    Função auxiliar que calcula cada um dos coeficientes de NTT(g).
    
    Parâmetros:
        - g : elemento para o qual vai ser calculado o coeficiente
        - i : indice o coeficiente de g que va i ser calculado
    '''
    def g_i(g, i) :
        g_i = 0
        for j in range(n) :
            g_i += psi**i * g[j] * w**(i*j)
        return g_i
    
    coefs = [0] * n
    for i in range(n) :
        coefs[i] = g_i(g,i)
        
    return RQ(coefs)

'''
Implementação da função NTT-1
'''
def ntt_inv(_g) :
    
    w, psi = 3844, 62 # Parâmetros definidos no documento da especificação do kyber
    
    '''
    Função auxiliar que calcula cada um dos coeficientes de g.
    
    Parâmetros:
        - g : elemento para o qual vai ser calculado o coeficiente
        - i : indice o coeficiente de g que va i ser calculado
    '''
    def _g_i(_g, i) :
        g_i = n**-1 * psi**-1
        soma = 0
        for j in range(n) :
            soma += int(_g[j]) * w**(-i*j)
        return int(g_i)*int(soma)
    
    coefs = [0] * n
    for i in range(n) :
        print('A calcular coeficiente g^', i)
        coefs[i] = _g_i(_g,i)
        
    return RQ(coefs)

O seguinte excerto de código permite-nos implementar a funcionalidade de **multiplicação** entre uma **matriz** e um **vetor**, ambos de elementos pertencentes a $R_q$, mais concretamente através da função **multMatrixVector()**. Esta necessita ainda de duas funções auxiliares, em que, uma delas implementa diretamente a multiplicação entre elementos pertencentes a $R_q$ (**pointwise_mult()**) e a outra permite-nos somar elementos pertencentes a $R_q$ ().

In [3]:
'''
Função que implementa a multiplicação entre duas 
entradas de vetores/matrizes de forma pointwise 
(coeficiente a coeficiente)

Parâmetros :
    - e1 e e2 : elemento/entrada da matriz/vetor
'''
def pointwise_mult(e1,e2) :
    
    mult_vector = []
    for i in range(n) :
        mult_vector.append(e1[i] * e2[i])
    return mult_vector

'''
Função que implementa a soma entre duas 
entradas de vetores/matrizes de forma pointwise 
(coeficiente a coeficiente)

Parâmetros :
    - e1 e e2 : elemento/entrada da matriz/vetor
'''
def pointwise_sum(e1,e2) :
    
    sum_vector = []
    for i in range(n) :
        sum_vector.append(e1[i] + e2[i])
    return sum_vector

'''
Função que retorna um vetor resultante da 
multiplicação entre uma matriz M e um vetor v
'''
def multMatrixVector(M,v,k) :
    T = NTT()
    As = []
    for i in range(k) :
        As.append([0] * n)
        for j in range(k) :
            As[i] = pointwise_sum(As[i],pointwise_mult(M[i][j], v[j]))
            #As[i] += T.ntt_inv(M[i][j] * v[j])
    return As


In [4]:
arr = [32,42,34,5,35,3,5,7,54,34,21,43,5,2,46,7,3,3,21,43,53,0,0,0,0,0,0,0,0,0,0,0,32,42,34,5,35,3,5,7,54,34,21,43,5,2,46,7,3,3,21,43,53,0,0,0,0,0,0,0,0,0,0,0]
poly = decode(2,arr)
print(len(poly.list()))
print(poly.list())
#print(len(encode(2,poly)))
print(arr)
print(encode(2,poly))
if arr == encode(2,poly) :
    print('\n[Success] Os arrays correspondem!')
else :
    print('\n[Erro] Algo correu mal com o decode/encode!')

256
[0, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0, 1, 1, 0, 0, 3, 0, 2, 0, 3, 0, 0, 0, 1, 1, 0, 0, 3, 1, 0, 0, 2, 1, 3, 0, 2, 0, 2, 0, 1, 1, 1, 0, 3, 2, 2, 0, 1, 1, 0, 0, 2, 0, 0, 0, 2, 3, 2, 0, 3, 1, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 1, 1, 1, 0, 3, 2, 2, 0, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0, 1, 1, 0, 0, 3, 0, 2, 0, 3, 0, 0, 0, 1, 1, 0, 0, 3, 1, 0, 0, 2, 1, 3, 0, 2, 0, 2, 0, 1, 1, 1, 0, 3, 2, 2, 0, 1, 1, 0, 0, 2, 0, 0, 0, 2, 3, 2, 0, 3, 1, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 1, 1, 1, 0, 3, 2, 2, 0, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 5, 2, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 5, 2, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[32, 42, 3

In [5]:
'''
Implementação da classe NTT
'''
class NTT(object):
#    
    def __init__(self, n=256):
        if not any([n == t for t in [32,64,128,256,512,1024,2048]]):
            raise ValueError("improper argument ",n)
        self.n = n  
        self.q = 2*n+1
        while True:
            if (self.q).is_prime():
                break
            self.q += 2*n
        #print(self.q)
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="xbar")
        w = (self.R).gen(); self.w = w
        
        g = (w^n + 1)
        #print(len(g.roots(multiplicities=False)))
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])  
    
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def ntt_inv(self,ff):                              ## transformada inversa
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)

In [6]:
# Teste

T = NTT()

f = T.random_pol(255)
#print(f)

ff = T.ntt(f)
#print(ff)

fff = T.ntt_inv(ff)

# print(fff)
print("Correto ? ",f == fff)

Correto ?  True


In [200]:
'''
Implementação da classe 
'''

class KyberPKE :
    
    def __init__(self,n,k,q,nn,du,dv,dt) :
        
        self.n = n
        self.k = k
        self.q = q
        self.nn = nn
        self.du = du
        self.dv = dv
        self.dt = dt
        Qq = PolynomialRing(GF(q), 'x')
        y = Qq.gen()
        RQ = QuotientRing(Qq, y^n+1)
        self.Rq = RQ
        
    '''
    Função que permite a geração de 
    uma chave pública
    '''
    def keygen(self) :
        d = os.urandom(32)
        h = G(d)
        digest = h.digest()
        ro,sigma = digest[:32], digest[32:]
        N = 0
        A, s, e = [], [], []
        # Construção da matriz A
        for i in range(self.k) :
            A.append([])
            for j in range(self.k) :
                xof = XOF()
                xof.update(ro + j.to_bytes(4,'little') + i.to_bytes(4,'little'))
                A[i].append(parse(xof.digest(int(q))))
        
        # Construção do vetor s
        for i in range(self.k) :
            prf = PRF()
            prf.update(sigma + int(N).to_bytes(4,'little'))
            s.append(cbd(self.nn, prf.digest(int(q+1))))
            N += 1
        # Construção do vetor e
        for i in range(self.k) :
            prf = PRF()
            prf.update(sigma + int(N).to_bytes(4,'little'))
            e.append(cbd(self.nn, prf.digest(int(q))))
            N += 1
        # Calculo do ntt de s
        T = NTT()
        _s = []
        for i in range(self.k) :
            _s.append(T.ntt(s[i]))
        
        t = multMatrixVector(A,_s,self.k)
        
        for i in range(self.k) :
            t[i] = T.ntt_inv(t[i])
            t[i] = self.Rq(pointwise_sum(t[i].list(),e[i].list()))
            
        #print(t[0].list())
        # compress(q,t,dt)
        # percorremos cada um dos polinomios do vetor t
        for i in range(self.k) :
            lst = t[i].list()
            for j in range(len(t[i].list())) :
                lst[j] = compress(self.q,lst[j],self.dt)
            t[i] = self.Rq(lst)
        
        # Aqui temos que t = compress(q,t,dt)
        # Calculamos agora pk = (encode(dt,t) || ro)
        pk = []
        for i in range(self.k) :
            res = encode(self.dt,t[i])
            #dec = decode(self.dt,res)
            #print(t[i].list()) ; print() ; print(dec.list())
            #print(len(t[i].list()))
            for j in range(len(res)) :
                pk.append(res[j])
        #print(len(pk))
        for i in range(len(ro)) :
            pk.append(ro[i])
        
        # print(len(pk))
        # Falta agora calcular sk = encode(13,s mod q)
        
        # Para cada polinomio :
        for i in range(self.k) :
            # Para cada coeficiente do polinomio :
            lst = _s[i]
            for j in range(len(_s[i])) :
                lst[j] = mod(_s[i][j],self.q)
            _s[i] = lst
        # agora tratamos do encode :
        sk = []
        for i in range(self.k) :
            res = encode(13,self.Rq(_s[i]))
            for bt in res :
                sk.append(bt)
        
        return(pk,sk)
        
        
    '''
    Função que implementa a cifragem de mensagens
        
    Parâmetros :
        - pk : Chave privada gerada
        - m : mensagem a ser cifrada
        - r : Random coins
    '''
    def encryption(self, pk, m, r) :
        
        '''
        Função auxiliar que permite transformar um 
        array de bytes (representados por integers) 
        em bytes (python)
        '''
        def byteArrToBytes(btArray) :
            byts = b''
            for i in btArray :
                byts += i.to_bytes(1,'little')
            return byts
        
        
        N = 0
        t = []
        
        # Implementação do decode(dt,pk)
        for i in range(self.k) :
            part = pk[i*32*self.dt:i*32*self.dt+32*self.dt]
            #print(len(part))
            t.append(decode(self.dt,part))
        # Implementação do decompress(q,decode(dt,pk),dt)
        for i in range(self.k) :
            lst = t[i].list()
            for j in range(len(t)) :
                lst[j] = decompress(self.q,lst[j],self.dt)
            t[i] = self.Rq(lst)
                
        ro = byteArrToBytes(pk[self.dt*self.k*self.n/8:])
        
        #print(ro)
        
        At = []
        # Construção da matriz A
        for i in range(self.k) :
            At.append([])
            for j in range(self.k) :
                xof = XOF()
                xof.update(ro + i.to_bytes(4,'little') + j.to_bytes(4,'little'))
                At[i].append(parse(xof.digest(int(self.q)))) 
        rr, e1 = [], []
        # Construção do vetor rr
        for i in range(self.k) :
            prf = PRF()
            prf.update(r + int(N).to_bytes(4,'little'))
            rr.append(cbd(self.nn, prf.digest(int(q+1))))
            N += 1
        # Construção do vetor e1
        for i in range(self.k) :
            prf = PRF()
            prf.update(r + int(N).to_bytes(4,'little'))
            e1.append(cbd(self.nn, prf.digest(int(self.q))))
            N += 1
        
        prf = PRF()
        prf.update(r + int(N).to_bytes(4,'little'))
        e2 = cbd(self.nn, prf.digest(int(self.q)))
        
        # Cálculo do ^rr :
        _rr = []
        T = NTT()
        for i in range(self.k) :
            _rr.append(T.ntt(rr[i]))
            
        # Cálculo do vetor em Rq u
        u = multMatrixVector(At,_rr,self.k)
        
        for i in range(self.k) :
            u[i] = T.ntt_inv(u[i])
            u[i] = self.Rq(pointwise_sum(u[i].list(),e1[i].list()))
            
        # Cálculo do v :
        v = [0] * self.n
            # Calculo de NTT(t) transposta :
        for i in range(self.k) :
            t[i] = T.ntt(t[i])

            # Calculo de v = NTT(t)T . _rr :
        for i in range(self.k) :
            v = pointwise_sum(v,pointwise_mult(t[i],_rr[i]))
            
            # Calculo de v = NTT-1(NTT(t)T . _rr)
        v = T.ntt_inv(v)
            # Calculo de v = NTT-1(NTT(t)T . _rr) + e2
        v = pointwise_sum(v.list(),e2.list())
        v = self.Rq(v)
        
            # Calculamos o decode(1,decompress(q,m,1))
        decompressed_m = []
        for i in range(len(m)) :
            decompressed_m.append(decompress(self.q, m[i], 1))
        
        decompressed_m = decode(1, decompressed_m)
        
            # Calculo do valor final de v:
        v = self.Rq(pointwise_sum(v.list(),decompressed_m.list()))
        
        # Cálculo de c1 :
        c1 = []
            # Calculo de compress(q,u,du) :
        for i in range(self.k) :
            lst = u[i].list()
            for j in range(len(u[i].list())) :
                lst[j] = compress(self.q,lst[j],self.du)
            u[i] = self.Rq(lst)
            # Calculo de encode(du,compress(q,u,du))
        for i in range(self.k) :
            u[i] = encode(self.du,u[i])
            for bt in u[i] :
                c1.append(bt)
                
        # Cálculo de c2 :
        
            # Calculo de compress(q,v,dv) :
        lst = v.list()
        for i in range(len(v.list())) :
            lst[i] = compress(self.q,lst[i],self.dv)
        v = self.Rq(lst)
            # Calculo de encode(dv,compress(q,v,dv)) :
        c2 = encode(self.dv,v)
        
        return c1+c2
    
    '''
    Função que implementa a decifragem de mensagens
    '''
    def decryption(self,sk,ct) :
        
        T = NTT()
        c1 = ct[:self.du*self.k*self.n/8]
        c2 = ct[self.du*self.k*self.n/8:]
        
        # Calculo de u = decompress(q,decode(du,ct),du):
        u = []
            # Calculo de decompress(q,decode(du,ct),du) :
        for i in range(self.k) :
            part = c1[i*32*self.du:i*32*self.du+32*self.du]
            u.append(decode(self.du,c1))
            lst = u[i].list()
            for j in range(len(u[i].list())) :
                lst[j] = decompress(self.q,lst[j],self.du)
            u[i] = self.Rq(lst)
                
        # Calculo de v :
        v = decode(self.dv,c2)
        lst = v.list()
        for i in range(len(v.list())) :
            lst[i] = decompress(self.q,lst[i],self.dv)
        v = self.Rq(lst)
            
        # Calculo de _s :
        _s = []
        for i in range(self.k) :
            _s.append(decode(13,sk[i*32*13:i*32*13+32*13]))
            
        # Calculo de m :
            
            # Calculo de NTT(u) :
        for i in range(self.k) :
            u[i] = T.ntt(u[i])    
            # Calculo de sT . NTT(u) :
        mult = self.Rq([])
        for i in range(self.k) :
            mult = pointwise_sum(mult,pointwise_mult(_s[i],u[i]))
            # Calculo de NTT-1(sT . NTT(u)) :
        mult = T.ntt_inv(mult)
            # Calculo de v - NTT-1(sT . NTT(u)) :
        dif = [0] * self.n
        for i in range(self.n) :
            dif[i] = v.list()[i]-mult.list()[i]
            # Calculo de m = compress(q,v - NTT-1(sT . NTT(u)),1)
        m = []
        for i in range(self.n) :
            m.append(compress(self.q,dif[i],1))
            
        m = encode(1,self.Rq(m))
        
        return m
        
            
            

In [201]:
k = KyberPKE(n=256,k=2,q=7681,nn=5,du=11,dv=3,dt=11)
(pk,sk) = k.keygen()

print('Tamanho da chave publica: ',len(pk))
#print('\nChave privada: ')
#print(sk)

m = [32,4,35,78,64,45,2,35,64,45,2,35,53,34,54,32,32,4,35,78,64,45,2,35,64,45,2,35,53,34,54,32]
print('Mensagem a cifrar: ',m) ; print()

ct = k.encryption(pk,m,os.urandom(32))

dct = k.decryption(sk,ct)

print('Mensagem decifrada: ',dct) ; print()

#print('Chave pública: ')
#print(pk)
#print('\nChave privada: ')
#print(sk)

Tamanho da chave publica:  736
Mensagem a cifrar:  [32, 4, 35, 78, 64, 45, 2, 35, 64, 45, 2, 35, 53, 34, 54, 32, 32, 4, 35, 78, 64, 45, 2, 35, 64, 45, 2, 35, 53, 34, 54, 32]

Mensagem decifrada:  [88, 192, 37, 247, 236, 125, 239, 206, 149, 160, 166, 214, 217, 222, 10, 166, 38, 130, 169, 7, 129, 50, 201, 47, 199, 136, 245, 111, 181, 221, 55, 5]

